In [3]:
import requests
import json
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()
from tqdm import tqdm


In [4]:
should_download_data = False

# Get Games
Fetch games and their id's for multiple months to collect data

In [5]:
# Set common variables
api_key = os.environ['SPORTRADAR_API_KEY']
access_level = 'trial'
version = 'v8'
language_code = 'en'
season_year = '2022'

In [6]:
if should_download_data:
    season_schedule = f"https://api.sportradar.com/nba/{access_level}/{version}/{language_code}/games/{season_year}/REG/schedule.json?api_key={api_key}"
    response = requests.get(season_schedule)
    if response.status_code == 200:
        try:
            schedule_data = response.json()
        except ValueError as e:
            print("Error decoding JSON:", e)
    else:
        print("Failed to retrieve data: ", response.status_code)
    schedule_data = response.json()
    print(json.dumps(schedule_data, indent=4, sort_keys=True))

In [7]:
if should_download_data: 
    with (open("data/season_schedule.json", "w")) as f:
        json.dump(schedule_data, f, indent=4)

In [8]:
# Open the file
with open("data/season_schedule.json", "r") as f:
    schedule_data = json.load(f)

flat_data = schedule_data['games']

df = pd.DataFrame(flat_data)
df.head()

,id,status,coverage,scheduled,home_points,away_points,track_on_court,sr_id,reference,time_zones,venue,broadcasts,home,away,title,neutral_site
0,9f01b268-29c5-4f0e-bf67-21e3dbcf3005,closed,full,2022-10-18T23:30:00Z,126.0,117.0,True,sr:match:35431579,0022200001,"{'venue': 'US/Eastern', 'home': 'US/Eastern', ...","{'id': '7d69b080-91ca-53c9-9302-45c1a72c5549',...","[{'network': 'TNT', 'type': 'TV', 'locale': 'N...","{'name': 'Boston Celtics', 'alias': 'BOS', 'id...","{'name': 'Philadelphia 76ers', 'alias': 'PHI',...",NaN,NaN
1,c665e441-9f38-48a7-8796-1f7292f97db1,closed,full,2022-10-19T02:00:00Z,123.0,109.0,True,sr:match:35431587,0022200002,"{'venue': 'US/Pacific', 'home': 'US/Pacific', ...","{'id': '938016dc-9e1d-4abc-88f5-3a7d772332e6',...","[{'network': 'TNT', 'type': 'TV', 'locale': 'N...","{'name': 'Golden State Warriors', 'alias': 'GS...","{'name': 'Los Angeles Lakers', 'alias': 'LAL',...",NaN,NaN
2,1f8105fe-e6ca-475f-aea6-15b455892a9e,closed,full,2022-10-19T23:00:00Z,107.0,114.0,True,sr:match:35431591,0022200004,"{'venue': 'US/Eastern', 'home': 'US/Eastern', ...","{'id': '24bb478e-eb31-5f8a-8c8d-07f513169ec1',...","[{'network': 'NBCS-DC', 'type': 'TV', 'locale'...","{'name': 'Indiana Pacers', 'alias': 'IND', 'id...","{'name': 'Washington Wizards', 'alias': 'WAS',...",NaN,NaN
3,9f64a3b0-46bd-4caa-b5c3-4578e9bab8d8,closed,full,2022-10-19T23:00:00Z,113.0,109.0,True,sr:match:35431589,0022200003,"{'venue': 'US/Eastern', 'home': 'US/Eastern', ...","{'id': '5a9ddefc-2267-4fd1-8d6e-0f82163ce8bd',...","[{'network': 'BSDET', 'type': 'TV', 'locale': ...","{'name': 'Detroit Pistons', 'alias': 'DET', 'i...","{'name': 'Orlando Magic', 'alias': 'ORL', 'id'...",NaN,NaN
4,44b31f4a-bc51-41aa-b1d9-a2bfc95d4e0d,closed,full,2022-10-19T23:30:00Z,108.0,105.0,True,sr:match:35431599,0022200008,"{'venue': 'US/Eastern', 'home': 'US/Eastern', ...","{'id': '62cc9661-7b13-56e7-bf4a-bba7ad7be8da',...","[{'network': 'BSOH', 'type': 'TV', 'locale': '...","{'name': 'Toronto Raptors', 'alias': 'TOR', 'i...","{'name': 'Cleveland Cavaliers', 'alias': 'CLE'...",NaN,NaN


In [9]:

if should_download_data:
    def get_play_by_play(game_id):
        return f"https://api.sportradar.com/nba/{access_level}/{version}/{language_code}/games/{game_id}/pbp.json?api_key={api_key}"


    # Get every game id from the season schedule
    game_ids = []
    for game in schedule_data['games']:
        game_ids.append(game['id'])

    # Get play by play data for each game, add it to a list, and write the list to a json file
    play_by_play_data = []
    for game_id in game_ids:
        response = requests.get(get_play_by_play(game_id))
        if response.status_code == 200:
            try:
                play_by_play_data.append(response.json())
            except ValueError as e:
                print("Error decoding JSON:", e)
        else:
            print("Failed to retrieve data: ", response.status_code)

    with (open("data/play_by_play_data.json", "w")) as f:
        json.dump(play_by_play_data, f, indent=4)

In [10]:
# Get data from the json file and put it into a dataframe

with open('data/play_by_play_data.json') as f:
    data = json.load(f)


In [79]:
# Remove duplicates from data
# TODO: Run the data download again, and remove this line
data = [i for n, i in enumerate(data) if i not in data[n + 1:]]

df = pd.DataFrame(data)
df.head()

,id,status,coverage,scheduled,duration,attendance,lead_changes,times_tied,clock,quarter,...,entry_mode,sr_id,clock_decimal,time_zones,home,away,periods,deleted_events,title,neutral_site
0,9f01b268-29c5-4f0e-bf67-21e3dbcf3005,closed,full,2022-10-18T23:30:00+00:00,2:31,19156.0,8.0,7.0,00:00,4.0,...,WEBSOCKET,sr:match:35431579,00:00,"{'venue': 'US/Eastern', 'home': 'US/Eastern', ...","{'name': 'Celtics', 'alias': 'BOS', 'market': ...","{'name': '76ers', 'alias': 'PHI', 'market': 'P...","[{'type': 'quarter', 'id': 'b2055122-2475-4178...",[{'id': '3a7522fd-dd18-4644-aa0f-fb8294b65c84'...,NaN,NaN
1,c665e441-9f38-48a7-8796-1f7292f97db1,closed,full,2022-10-19T02:00:00+00:00,2:33,18064.0,1.0,2.0,00:00,4.0,...,WEBSOCKET,sr:match:35431587,00:00,"{'venue': 'US/Pacific', 'home': 'US/Pacific', ...","{'name': 'Warriors', 'alias': 'GSW', 'market':...","{'name': 'Lakers', 'alias': 'LAL', 'market': '...","[{'type': 'quarter', 'id': '30829330-c8e3-4ff6...",[{'id': '625bb81a-0ede-4a8e-8f16-4cc5cc2704ad'...,NaN,NaN
2,1f8105fe-e6ca-475f-aea6-15b455892a9e,closed,full,2022-10-19T23:00:00+00:00,2:22,15027.0,0.0,0.0,00:00,4.0,...,WEBSOCKET,sr:match:35431591,00:00,"{'venue': 'US/Eastern', 'home': 'US/Eastern', ...","{'name': 'Pacers', 'alias': 'IND', 'market': '...","{'name': 'Wizards', 'alias': 'WAS', 'market': ...","[{'type': 'quarter', 'id': 'e5c8501a-729a-4746...",[{'id': 'c36853e2-f892-42f7-a482-2bf4209d139a'...,NaN,NaN
3,9f64a3b0-46bd-4caa-b5c3-4578e9bab8d8,closed,full,2022-10-19T23:00:00+00:00,2:16,20190.0,7.0,4.0,00:00,4.0,...,WEBSOCKET,sr:match:35431589,00:00,"{'venue': 'US/Eastern', 'home': 'US/Eastern', ...","{'name': 'Pistons', 'alias': 'DET', 'market': ...","{'name': 'Magic', 'alias': 'ORL', 'market': 'O...","[{'type': 'quarter', 'id': 'f2bf467f-85da-4a05...",[{'id': '50061ec0-25f9-4ab2-af46-8867d9565cc0'...,NaN,NaN
4,44b31f4a-bc51-41aa-b1d9-a2bfc95d4e0d,closed,full,2022-10-19T23:30:00+00:00,2:19,19800.0,16.0,17.0,00:00,4.0,...,WEBSOCKET,sr:match:35431599,00:00,"{'venue': 'US/Eastern', 'home': 'US/Eastern', ...","{'name': 'Raptors', 'alias': 'TOR', 'market': ...","{'name': 'Cavaliers', 'alias': 'CLE', 'market'...","[{'type': 'quarter', 'id': '08bea6a0-f82a-41a9...",[{'id': '16d692db-a7a7-43fc-9535-a3187cd349ab'...,NaN,NaN


In [122]:
def process_data(df):
    df_copy = df.copy()

    # List of columns to drop initially
    columns_to_drop = ["track_on_court", "deleted_events", "title", "neutral_site", 'coverage', 'scheduled', "status", "attendance", "clock", "duration", "reference", "entry_mode", "sr_id", "clock_decimal", "time_zones"]

    # List of team features to drop
    team_features_to_drop = ["name", "alias", "market", "id", "sr_id", "bonus", "remaining_timeouts", "reference", "points"]

    # Process home and away data
    for team in ['home', 'away']:
        team_data = pd.json_normalize(df_copy[team]).drop(team_features_to_drop, axis=1)
        team_data.columns = [f'{team}_{col}' for col in team_data.columns]
        df_copy = df_copy.join(team_data)

    # Create features for home win percentage and away win percentage based on the team's record
    for team in ['home', 'away']:
        df_copy[f'{team}_win_pct'] = df_copy[f"{team}_record.wins"] / (df_copy[f"{team}_record.wins"] + df_copy[f"{team}_record.losses"])
        columns_to_drop.extend([f"{team}_record.wins", f"{team}_record.losses"])

    # Drop the home, away and record columns from the original dataframe
    columns_to_drop.extend(["home", "away"])
    df_copy.drop(columns_to_drop, axis=1, inplace=True)

    return df_copy

df_processed = process_data(df)
df_processed.head()

,id,lead_changes,times_tied,quarter,periods,home_win_pct,away_win_pct
0,9f01b268-29c5-4f0e-bf67-21e3dbcf3005,8.0,7.0,4.0,"[{'type': 'quarter', 'id': 'b2055122-2475-4178...",0.758621,0.555556
1,c665e441-9f38-48a7-8796-1f7292f97db1,1.0,2.0,4.0,"[{'type': 'quarter', 'id': '30829330-c8e3-4ff6...",0.482759,0.407407
2,1f8105fe-e6ca-475f-aea6-15b455892a9e,0.0,0.0,4.0,"[{'type': 'quarter', 'id': 'e5c8501a-729a-4746...",0.430380,0.435897
3,9f64a3b0-46bd-4caa-b5c3-4578e9bab8d8,7.0,4.0,4.0,"[{'type': 'quarter', 'id': 'f2bf467f-85da-4a05...",0.266667,0.310345
4,44b31f4a-bc51-41aa-b1d9-a2bfc95d4e0d,16.0,17.0,4.0,"[{'type': 'quarter', 'id': '08bea6a0-f82a-41a9...",0.464286,0.620690


In [116]:
df_processed["periods"][0][0]["events"][0]

{'id': '03c554f3-62e6-4746-82f5-d0b8f1733c20',
 'clock': '12:00',
 'updated': '2022-10-18T23:35:25Z',
 'description': '76ers lineup change (Joel Embiid, Tyrese Maxey, James Harden, Tobias Harris, P.J. Tucker)',
 'wall_clock': '2022-10-18T23:35:25Z',
 'sequence': 1666136125393,
 'home_points': 0,
 'away_points': 0,
 'clock_decimal': '12:00',
 'number': 5002,
 'event_type': 'lineupchange',
 'attribution': {'name': '76ers',
  'market': 'Philadelphia',
  'id': '583ec87d-fb46-11e1-82cb-f4ce4684ea4c',
  'sr_id': 'sr:team:3420',
  'reference': '1610612755'},
 'on_court': {'home': {'name': 'Celtics',
   'market': 'Boston',
   'id': '583eccfa-fb46-11e1-82cb-f4ce4684ea4c',
   'sr_id': 'sr:team:3422',
   'reference': '1610612738',
   'players': [{'full_name': 'Jaylen Brown',
     'jersey_number': '7',
     'id': 'ffc5579c-783f-4d62-80ab-3c3dcb05a27d',
     'sr_id': 'sr:player:996297',
     'reference': '1627759'},
    {'full_name': 'Marcus Smart',
     'jersey_number': '36',
     'id': 'af460d21-

In [139]:
import pandas as pd

def extract_periods(df):
    # Create an empty DataFrame to store the extracted data
    extracted_data = pd.DataFrame()

    features_to_keep = ['game_id','type', 'id', 'number', 'sequence', 'times_tied',
       'lead_changes', "events"]

    for i in range(len(df)):
        # Iterate through each game entry
        game_id = df['id'].iloc[i]
        periods = df['periods'].iloc[i]

        for period in periods:
            # For each period, flatten the period data and the nested 'scoring' data
            period_data = pd.json_normalize(period)
            scoring_data = pd.json_normalize(period['scoring'])

            # Combine period data with scoring data
            combined_data = pd.concat([period_data, scoring_data], axis=1)

            # Add game-level information (e.g., game ID)
            combined_data['game_id'] = game_id

            # Append to the extracted_data DataFrame
            extracted_data = pd.concat([extracted_data, combined_data], ignore_index=True)

    # Keep only the columns we want
    extracted_data = extracted_data[features_to_keep]

    return extracted_data

# Use the function to extract period data
period_data = extract_periods(df_processed)
# Display the first few rows
period_data.head()

,game_id,type,id,number,sequence,times_tied,lead_changes,events
0,9f01b268-29c5-4f0e-bf67-21e3dbcf3005,quarter,b2055122-2475-4178-8643-87dc7c940e07,1,1,3.0,3,[{'id': '03c554f3-62e6-4746-82f5-d0b8f1733c20'...
1,9f01b268-29c5-4f0e-bf67-21e3dbcf3005,quarter,54baa13c-4695-4c15-9a44-86e46a547d58,2,2,3.0,4,[{'id': 'd4e131e5-05b8-4ba4-9a19-f7032abfd7e1'...
2,9f01b268-29c5-4f0e-bf67-21e3dbcf3005,quarter,dbb8fe7d-c9c4-4a30-a1a4-63ce1dc24dea,3,3,1.0,1,[{'id': '49815faa-6f74-474a-acbc-6c4379014ea1'...
3,9f01b268-29c5-4f0e-bf67-21e3dbcf3005,quarter,6a988dec-8692-4717-996a-e468b8eba2f0,4,4,0.0,0,[{'id': 'dfc4f1b1-fb06-4ac7-ab39-2ebcaf1b8c3d'...
4,c665e441-9f38-48a7-8796-1f7292f97db1,quarter,30829330-c8e3-4ff6-8e23-811d3a33e1c1,1,1,2.0,1,[{'id': '9183a93e-e763-4db8-a508-c80be8545c35'...


In [180]:
from tqdm import tqdm

def extract_events(df): 
    extracted_data = pd.DataFrame()

    features_to_keep = ['id', 'number', 'event_type', 'home_points', 'away_points', 'clock_decimal', "on_court.home.id" ,'on_court.home.players',
       "on_court.away.id", 'on_court.away.players', 'game_id', 'possession.id', 'statistics', 'attempt']

    # Add tqdm() around the range to create a progress bar
    for i in tqdm(range(len(df))):
        game_id = df['game_id'].iloc[i]
        events = df['events'].iloc[i]

        for event in events:
            event_data = pd.json_normalize(event)
            event_data['game_id'] = game_id
            extracted_data = pd.concat([extracted_data, event_data], ignore_index=True)

    print(extracted_data.columns)
    extracted_data = extracted_data[features_to_keep]
    return extracted_data

event_data = extract_events(period_data[:10])


100%|██████████| 10/10 [00:01<00:00,  6.66it/s]

Index(['id', 'clock', 'updated', 'description', 'wall_clock', 'sequence',
       'home_points', 'away_points', 'clock_decimal', 'number', 'event_type',
       'attribution.name', 'attribution.market', 'attribution.id',
       'attribution.sr_id', 'attribution.reference', 'on_court.home.name',
       'on_court.home.market', 'on_court.home.id', 'on_court.home.sr_id',
       'on_court.home.reference', 'on_court.home.players',
       'on_court.away.name', 'on_court.away.market', 'on_court.away.id',
       'on_court.away.sr_id', 'on_court.away.reference',
       'on_court.away.players', 'game_id', 'attribution.team_basket',
       'possession.name', 'possession.market', 'possession.id',
       'possession.sr_id', 'possession.reference', 'statistics',
       'location.coord_x', 'location.coord_y', 'location.action_area',
       'turnover_type', 'qualifiers', 'attempt', 'duration'],
      dtype='object')


In [181]:
# Display head where "qualifiers" column where value is not null

event_data[event_data['statistics'].notnull()].head()   


,id,number,event_type,home_points,away_points,clock_decimal,on_court.home.id,on_court.home.players,on_court.away.id,on_court.away.players,game_id,possession.id,statistics,attempt
3,6fe0abb4-7cd5-4676-8f98-5a98a790048c,7,twopointmiss,0,0,11:38,583eccfa-fb46-11e1-82cb-f4ce4684ea4c,"[{'full_name': 'Jaylen Brown', 'jersey_number'...",583ec87d-fb46-11e1-82cb-f4ce4684ea4c,"[{'full_name': 'Joel Embiid', 'jersey_number':...",9f01b268-29c5-4f0e-bf67-21e3dbcf3005,NaN,"[{'type': 'fieldgoal', 'made': False, 'shot_ty...",NaN
4,3eedb0e6-f4ce-4752-8ca7-c0bcbefdb61f,9,rebound,0,0,11:35,583eccfa-fb46-11e1-82cb-f4ce4684ea4c,"[{'full_name': 'Jaylen Brown', 'jersey_number'...",583ec87d-fb46-11e1-82cb-f4ce4684ea4c,"[{'full_name': 'Joel Embiid', 'jersey_number':...",9f01b268-29c5-4f0e-bf67-21e3dbcf3005,583ec87d-fb46-11e1-82cb-f4ce4684ea4c,"[{'type': 'rebound', 'rebound_type': 'offensiv...",NaN
5,20f74c6a-f1eb-4283-b765-a65c8551f7f6,10,turnover,0,0,11:35,583eccfa-fb46-11e1-82cb-f4ce4684ea4c,"[{'full_name': 'Jaylen Brown', 'jersey_number'...",583ec87d-fb46-11e1-82cb-f4ce4684ea4c,"[{'full_name': 'Joel Embiid', 'jersey_number':...",9f01b268-29c5-4f0e-bf67-21e3dbcf3005,583eccfa-fb46-11e1-82cb-f4ce4684ea4c,"[{'type': 'turnover', 'team': {'name': '76ers'...",NaN
6,836add5b-8258-4f7e-a6b6-353c5a389e2a,11,twopointmade,2,0,11:15,583eccfa-fb46-11e1-82cb-f4ce4684ea4c,"[{'full_name': 'Jaylen Brown', 'jersey_number'...",583ec87d-fb46-11e1-82cb-f4ce4684ea4c,"[{'full_name': 'Joel Embiid', 'jersey_number':...",9f01b268-29c5-4f0e-bf67-21e3dbcf3005,583ec87d-fb46-11e1-82cb-f4ce4684ea4c,"[{'type': 'fieldgoal', 'made': True, 'shot_typ...",NaN
7,0bd9b470-1ed7-4d19-a0c1-0e78823ff6df,12,twopointmiss,2,0,11:05,583eccfa-fb46-11e1-82cb-f4ce4684ea4c,"[{'full_name': 'Jaylen Brown', 'jersey_number'...",583ec87d-fb46-11e1-82cb-f4ce4684ea4c,"[{'full_name': 'Joel Embiid', 'jersey_number':...",9f01b268-29c5-4f0e-bf67-21e3dbcf3005,NaN,"[{'type': 'fieldgoal', 'made': False, 'shot_ty...",NaN
